

---

 **#STEP 1:
Read and statistic data**








*#Kết nối tới file data ở github*

In [ ]:
import pandas as pd
import numpy as np
!git clone 'https://github.com/Phuc-ph-2109/doands102.git'

*#Đọc dữ liệu và thống kê*



In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
df = pd.read_csv("/content/doands102/vnexpress.csv")
#visualize số nhãn mỗi loại tập đầy đủ
print(Counter(np.array(df['label'])))
label = ["positive", "negative", "neutral"]
count_label = [555, 530, 555]
plt.bar(label, count_label, color = "green")
plt.title("COUNT_LABEL_full")
plt.xlabel("Label")
plt.ylabel("Number")
plt.show()

*#Chia tập train test, và Visualize nhãn*

In [ ]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,0].values
Y = df.iloc[:,1].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size =0.8, random_state = 0, shuffle = True)

#visualize số nhãn mỗi loại tập train
print(Counter(np.array(Y_train)))
count_label_train = [443, 429, 440]
plt.bar(label, count_label_train, color = "pink")
plt.title("Count Label Train")
plt.xlabel("Label")
plt.ylabel("Number")
plt.show()
#visualize số nhãn mỗi loại tập test
print(Counter(np.array(Y_test)))
count_label_test = [112, 101, 115]
plt.bar(label, count_label_test, color = "blue")
plt.title("Count Label Test")
plt.xlabel("Label")
plt.ylabel("Number")
plt.show()

**#STEP 2: Preprocessing data**

*# Xóa kí tự đặc biệt, tách từ theo ngôn ngữ Việt*

In [ ]:
# cài đặt thư viện pyvi
!pip install pyvi

In [32]:
from pyvi import ViTokenizer, ViPosTagger
import string
import itertools
def process_text(text):
    
    #1 Xóa kí tự đặc biệt, chuyển về chữ thường
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    #2 Gộp các kí tự trùng lặp
    nopunc = ''.join(ch for ch, _ in itertools.groupby(nopunc))
    #3 Vitokennizer (Tách từ theo từ điển việt nam (độ chính xác 80%))
    review = ViTokenizer.tokenize(nopunc)
    return review

*#Tạo bags of Word, sử dụng TfidfTransformer để giảm những từ xuất hiện nhiều nhưng vô nghĩa*
*Sử dụng pipeline kết hợp*

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# Model 1 xử dụng MultinomialNB của Naive Bayes
model1 = Pipeline([('vect', CountVectorizer(analyzer=process_text)),('tfidf', TfidfTransformer()),('clf1', MultinomialNB()),])

# Model 2 xử dụng Suport Vector Machine
model2 = Pipeline([('vect', CountVectorizer(analyzer=process_text)),('tfidf', TfidfTransformer()),('clf2', SVC()),])


**#STEP 3: Training model & Save model**

In [35]:
import pickle
# Model 1
model1.fit(X_train,Y_train)
filename = 'NaiveBayes_model.sav'
pickle.dump(model1, open(filename, 'wb'))
# Model 2
model2.fit(X_train,Y_train)
filename = 'SVM_model.sav'
pickle.dump(model2, open(filename, 'wb'))

**#STEP 4: Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
#Test set model 1
cm = confusion_matrix(Y_test, model1.predict(X_test))
plt.title('Confusion matrix on Test set Model 1:')
sns.heatmap(cm, annot = True, fmt = 'g', cmap = 'OrRd', annot_kws = {"size": 10})
plt.savefig('confusionmatrix_test_Logistic.png')
plt.show()

#Test set model 2
cm = confusion_matrix(Y_test, model2.predict(X_test))
plt.title('Confusion matrix on Test set Model 2:')
sns.heatmap(cm, annot = True, fmt = 'g', cmap = 'OrRd', annot_kws = {"size": 10})
plt.savefig('confusionmatrix_test_SVM.png')
plt.show()

**#STEP 5: Review the model**

In [ ]:
from sklearn.metrics import classification_report
#Model 1
print(classification_report(Y_test, model1.predict(X_test)))

In [ ]:
#Model 2
print(classification_report(Y_test, model2.predict(X_test)))

In [ ]:
#Compare model 1
df_compare = pd.DataFrame()
df_compare['cmt'] = X_test
df_compare['label'] = Y_test
df_compare['Predict label'] = model1.predict(X_test)
print('Compare on 20 line in test set:')
print(df_compare.head(20))

df_compare.to_excel('predict_NaiveBayes.xlsx')

In [ ]:
#Compare model 2
df_compare = pd.DataFrame()
df_compare['cmt'] = X_test
df_compare['label'] = Y_test
df_compare['Predict label'] = model2.predict(X_test)
print('Compare on 20 line in test set:')
print(df_compare.head(20))

df_compare.to_excel('predict_SVM.xlsx')